

<p>-Prepares FastText Twitter Embeddings for Transformer model</p>

<style>
td {
  text-align: center;
}

th {
  text-align: center;
}
</style>

## Setup

The following codes ensures the words embeddings have the same result after being randomly initialized

In [1]:
import tensorflow as tf
import numpy as np




In [2]:

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Navigate to the Directory Containing the CSV File
import os
os.chdir('/content/drive/My Drive/DS266 Project')

Begin by installing [TensorFlow Datasets](https://tensorflow.org/datasets) for loading the dataset and [TensorFlow Text](https://www.tensorflow.org/text) for text preprocessing:

In [4]:
# Install the most re version of TensorFlow to use the improved
# masking support for `tf.keras.layers.MultiHeadAttention`.
!apt install --allow-change-held-packages libcudnn8=8.1.0.77-1+cuda11.2 -q
!pip uninstall -y -q tensorflow keras tensorflow-estimator tensorflow-text -q
!pip install -q protobuf~=3.20.3
!pip install -q tensorflow_datasets
!pip install -q -U tensorflow-text tensorflow

Reading package lists...
Building dependency tree...
Reading state information...
Package libcudnn8 is not available, but is referred to by another package.
This may mean that the package is missing, has been obsoleted, or
is only available from another source

E: Version '8.1.0.77-1+cuda11.2' for 'libcudnn8' was not found
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 83.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 76.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 90.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 56.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 25.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This beh

In [5]:
# Install compatible TensorFlow version
!pip uninstall tensorflow -y -q
!pip install tensorflow==2.15 -q

# Reinstall tf-keras to resolve any potential dependency conflicts
!pip uninstall tf-keras -y -q
!pip install tf-keras -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 53.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 79.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 36.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 65.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-text 2.16.1 requires tensorflow<2.17,>=2.16.1; platform_machine != "arm64" or platform_system != "Darwin", but you have tensorflow 2.15.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 10.3 MB/s eta 0:00:00


In [6]:
!pip install tensorflow==2.15 -q #it needs it again for some reason

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-text 2.16.1 requires tensorflow<2.17,>=2.16.1; platform_machine != "arm64" or platform_system != "Darwin", but you have tensorflow 2.15.0 which is incompatible.
tf-keras 2.16.0 requires tensorflow<2.17,>=2.16, but you have tensorflow 2.15.0 which is incompatible.


In [7]:
!pip uninstall tensorflow-text -y -q
!pip install tensorflow-text==2.15 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 27.4 MB/s eta 0:00:00


In [8]:
!pip install gensim #needed to import fast text twitter embeddings

Import the necessary modules:

In [10]:
import logging
import time

import numpy as np
import matplotlib.pyplot as plt

import tensorflow_datasets as tfds
import tensorflow as tf

import tensorflow_text as text
from tensorflow import keras

from gensim.models import KeyedVectors

## Data handling



### Download the dataset

read the data

In [11]:
# Read the CSV File
import pandas as pd
load_data = 'train_enhanced_cleaned.csv'
tweets_df = pd.read_csv(load_data).drop_duplicates()
tweets_df.tail(3)

,text,target
7610,location m1 94 0104 utc5km s of volcano hawa...,1
7611,location police investigating after an ebike...,1
7612,location the latest more homes razed by nort...,1


###Load FastText and Setup

The “fasttext_english_twitter_100d.vec” file is a set of word vectors trained using the FastText library.

FastText is a library developed by Facebook’s AI Research (FAIR) lab. It’s dedicated to text classification and learning word representations. FastText models can be trained on more than a billion words on any multicore CPU in less than a few minutes.

Here’s a general process of how FastText trains word vectors:

Corpus Selection: FastText trains word vectors on large text corpora. For example, FastText provides pre-trained word vectors trained, in this case, a collection of English tweets.
Model Training: These models are trained using the Continuous Bag of Words (CBOW) model with position-weights, in a specified dimension (like 300), with character n-grams of length 5, a window of size 5, and 10 negatives2.
Vector Generation: Each line of the output file contains a word followed by its vectors, like in the default FastText text format. Each value is space-separated. Words are ordered by descending frequency.




In [12]:
# Load vectors directly from the file
vector_model = KeyedVectors.load_word2vec_format('fasttext_english_twitter_100d.vec')

After downloading Fast Text Tweet Embeddings we did a quick sanity check to see if some of the words we anticipate to commonly show up in our tweets text are captured.

In [13]:
def check_word(item_value, model):
  word = None
  #print(model[item_value])
  #print(item_value)
  try:
    if model[item_value].any():
      word = "\""+item_value +"\" exists in embeddings dictionary"
      vector = model[item_value]
  except:
    word = "\""+item_value +"\" does not exist"
    vector = np.zeros((100,))
  return word, vector

check = ['fire', 'hurricane', 'earthquake', 'sunami', 'disaster']
for i in range(len(check)-0):
  word, vector = check_word(check[i], vector_model)
  print(word)
  print(vector.shape)

"fire" exists in embeddings dictionary
(100,)
"hurricane" exists in embeddings dictionary
(100,)
"earthquake" exists in embeddings dictionary
(100,)
"sunami" does not exist
(100,)
"disaster" exists in embeddings dictionary
(100,)


### Set up the tokenizer

Now that you have loaded the dataset, you need to tokenize the text, so that each element is represented as a [token](https://developers.google.com/machine-learning/glossary#token) or token ID (a numeric representation).

Tokenization is the process of breaking up text, into "tokens". Depending on the tokenizer, these tokens can represent sentence-pieces, words, subwords, or characters. To learn more about tokenization, visit [this guide](https://www.tensorflow.org/text/guide/tokenizers).

In [14]:
sentences = tweets_df['text']
labels = tweets_df['target']
sentences.shape

(7066,)

In [15]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Instantiate the tokenizer
tokenizer = Tokenizer()

# Fit the tokenizer on your text data
tokenizer.fit_on_texts(sentences)


####MAX_TOKENS

In [17]:
MAX_TOKENS =30

In [18]:

# Convert sentences to sequences
sequences = tokenizer.texts_to_sequences(sentences) #sequences needed for batching set up

# Pad sequences
sequences = pad_sequences(sequences, maxlen=MAX_TOKENS, padding='post') #sequences needed for batching set up



####convert back to a word

In [19]:
#
tokenized_sentences = []
for sequence in sequences:
    words = [tokenizer.index_word[token] for token in sequence if token in tokenizer.index_word]
    sentence = ' '.join(words)
    tokenized_sentences.append(sentence)

###Tokenize and convert to FastText Twitter Embedding

####Vector Dim
vector_dim must be equal to the hyperparameter of d_model

In [20]:
vector_dim = 100 #equal to hyperparameter d_model

In [21]:
# Get the vocab size
vocab_size = len(tokenizer.word_index) + 1  # +1 for the padding token

# Initialize the embedding matrix with zeros
embedding_matrix = np.zeros((vocab_size, vector_dim))

# Fill the embedding matrix with vectors from the FastText model
for word, index in tokenizer.word_index.items():
    if word in vector_model:
        embedding_vector = vector_model[word] #FastText Twitter Embeddings
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector

# Convert the embedding matrix to a DataFrame
embedding_df = pd.DataFrame(embedding_matrix)



In [22]:
embedding_df.shape

(17984, 100)

####Write to file

In [26]:

# Save the DataFrame as a CSV file
embedding_df.to_csv('fast_text_twitter_embeddings.csv', index=False)

###Checking the Vectorization

Data Checks

In [27]:
loaded_embeddings_df = pd.read_csv('fast_text_twitter_embeddings.csv')

In [28]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer




# Tokenize a word and get its token ID
word_to_test = 'house'
token_id = tokenizer.word_index[word_to_test]

# Load the corresponding vector from the CSV
vector_from_csv = loaded_embeddings_df.iloc[token_id - 1].values  # -1 because indexing starts at 0

# Print the vector to verify
print(f"Vector for the word '{word_to_test}' from CSV:", vector_from_csv)


Vector for the word 'house' from CSV: [ 0.12065     0.14212     0.17784999  0.18043999 -0.21229     0.30074999
 -0.076699   -0.23617999 -0.33881     0.26030999 -0.41927001  0.81598997
  0.38793999  0.34344     0.40472001 -0.22395    -0.91606998 -0.061257
 -0.56077999  0.11166    -0.047045   -0.22796001 -0.21043     0.26730001
  0.058979    0.37796    -0.067553   -0.021946    0.069971    0.035356
 -0.43852001 -0.055163   -0.26499999 -0.20479     0.44907999 -0.042062
 -0.35530001  0.51872998  0.30981001  0.0067503  -0.46794999  0.50919998
 -0.38045001  0.010086    0.013727   -0.056027    0.07452     0.36842
  0.028126   -0.14568    -0.47525001  0.15092     0.033576    0.33785
  0.32330999 -0.26062    -0.38325    -0.14787     0.18810999  0.87696999
  0.30160999 -0.081211    0.18231    -0.062108    0.92413998  0.31123999
 -0.71069998  0.052805   -0.39974999 -0.32308     0.085745    0.43895
  0.34470001  0.3231      0.17323001  0.082239    0.02889    -0.18970001
  0.13151     0.09467    -0.

In [22]:

vector_model = KeyedVectors.load_word2vec_format('fasttext_english_twitter_100d.vec')



####Ensure the word is in the tokenizer's vocabulary

In [29]:

# Ensure the word is in the tokenizer's vocabulary
word_to_test = 'house'
if word_to_test in tokenizer.word_index:
    token_id = tokenizer.word_index[word_to_test]
else:
    raise ValueError(f"'{word_to_test}' not found in tokenizer's vocabulary.")


vector_from_csv = loaded_embeddings_df.iloc[token_id].values

# Get the vector for the word from the FastText model
if word_to_test in vector_model:
    vector_from_model = vector_model[word_to_test]
else:
    raise ValueError(f"Vector for '{word_to_test}' not found in FastText model.")

# Compare the two vectors
are_vectors_equal = np.allclose(vector_from_csv, vector_from_model, atol=1e-9)

# Print the result of the comparison
print(f"Does the vector for '{word_to_test}' from CSV match the FastText model? {are_vectors_equal}")

Does the vector for 'house' from CSV match the FastText model? True


visual verification

In [30]:
type(vector_from_model), vector_from_model.shape, vector_from_model

(numpy.ndarray,
 (100,),
 array([ 4.5484e-01,  1.1877e-01, -6.7182e-01,  4.0004e-02,  2.9812e-01,
         1.3266e-02,  2.2786e-01, -2.8099e-01, -4.4837e-02,  2.1004e-01,
         7.1316e-02,  2.0716e-01,  6.1164e-01,  2.0849e-02,  7.6408e-02,
         1.8154e-01, -5.7663e-01,  1.4443e-01, -5.2093e-02,  1.3182e-01,
         1.3530e-01,  1.5884e-01,  4.9659e-01,  8.6108e-02,  5.4261e-01,
         2.9661e-01, -1.9361e-01, -1.5841e-01,  9.3623e-02, -3.1574e-01,
         1.6179e-01,  4.0957e-01, -1.6413e-02,  9.4639e-02,  1.5520e-01,
        -2.4322e-02, -2.4437e-01,  3.2023e-04,  6.7701e-02,  4.5904e-01,
         4.2513e-02,  2.0157e-01,  2.9248e-01, -2.3573e-01,  1.3197e-01,
         1.9594e-02, -4.6181e-01,  1.2979e-01, -5.8775e-01, -1.6625e-01,
         9.8981e-01,  1.5905e-01, -6.1933e-02, -1.0294e-01, -3.5458e-03,
        -2.7351e-01, -2.3616e-01,  7.1419e-02,  4.0173e-01,  5.4632e-01,
         1.4278e-01, -1.8228e-01, -2.2210e-01, -8.3939e-01,  6.5804e-01,
         7.0661e-01, -7.13

In [31]:
type(vector_from_csv), vector_from_csv.shape, vector_from_csv

(numpy.ndarray,
 (100,),
 array([ 4.54840004e-01,  1.18770003e-01, -6.71819985e-01,  4.00040001e-02,
         2.98119992e-01,  1.32660000e-02,  2.27860004e-01, -2.80990005e-01,
        -4.48370017e-02,  2.10040003e-01,  7.13159963e-02,  2.07159996e-01,
         6.11639977e-01,  2.08490007e-02,  7.64079988e-02,  1.81539997e-01,
        -5.76629996e-01,  1.44429997e-01, -5.20929992e-02,  1.31819993e-01,
         1.35299996e-01,  1.58840001e-01,  4.96589988e-01,  8.61079991e-02,
         5.42609990e-01,  2.96609998e-01, -1.93609998e-01, -1.58409998e-01,
         9.36229974e-02, -3.15739989e-01,  1.61789998e-01,  4.09570009e-01,
        -1.64129995e-02,  9.46390033e-02,  1.55200005e-01, -2.43219994e-02,
        -2.44369999e-01,  3.20229999e-04,  6.77009970e-02,  4.59039986e-01,
         4.25130017e-02,  2.01570004e-01,  2.92479992e-01, -2.35730007e-01,
         1.31970003e-01,  1.95940007e-02, -4.61809993e-01,  1.29789993e-01,
        -5.87750018e-01, -1.66250005e-01,  9.89809990e-01,  1.5

Additional verification

####Load a Vector and Return a Word

In [32]:
from sklearn.neighbors import NearestNeighbors




# Define the function to find the closest word for a given vector
def find_closest_word(vector, nn_model, tokenizer):
    # Reshape the vector to match the input shape for the model (1, number of features)
    vector = vector.reshape(1, -1)

    # Find the nearest neighbor (closest vector in the space)
    _, indices = nn_model.kneighbors(vector)

    # Get the index of the closest vector
    closest_index = indices[0][0]

    # Find the corresponding word from the tokenizer
    for word, index in tokenizer.word_index.items():
        if index == closest_index:
            return word
    return "Word not found"




In [33]:
# Assuming 'loaded_embeddings_df' contains all the vectors from the CSV
# and 'tokenizer' is a dictionary mapping words to their indices

# Convert the embeddings dataframe to a numpy array
embeddings = loaded_embeddings_df.to_numpy()

# Create and fit the NearestNeighbors model
nn_model = NearestNeighbors(n_neighbors=1, algorithm='ball_tree')
nn_model.fit(embeddings)
# Example usage:
# Assuming 'vector_from_csv' is the vector you want to find the word for
word = find_closest_word(vector_from_csv, nn_model, tokenizer)
print(f"The word for the given vector is: {word}")

The word for the given vector is: house


#End of File
At this point the embeddings for loading to the transformer have been created